In [1]:
import dask.dataframe as pd
import dask

from dask.distributed import Client
from datetime import datetime

# client = Client(n_workers=8)

# CADES configuration
from dask_jobqueue import SLURMCluster
cluster = SLURMCluster(project='birthright', queue='high_mem_cd', cores=30, memory='150 GB', job_extra=["-N 1"], interface="ib0")
client = Client(cluster)

In [2]:
def to_date(d):
    if ":" == d[-3:-2]:
        d = d[:-3]+d[-2:]
    return d

In [3]:
patterns = pd.read_csv("data/safegraph/weekly-patterns/*patterns*", dtype={'naics_code': 'float64'}).set_index('safegraph_place_id')
patterns['date'] = patterns['date_range_start'].apply(lambda x: datetime.strptime(to_date(x), '%Y-%m-%dT%H:%M:%S%z').date(), meta=('date_range_start', 'object'))

In [13]:
patterns_grouped = patterns.groupby(['region', 'date', 'naics_4']).raw_visit_counts.sum().reset_index()

,region,date,naics_4,raw_visit_counts
npartitions=1,,,,
,object,object,object,int64
,...,...,...,...


In [6]:
essential = pd.read_csv("data/reference/essential_services_unique.csv", dtype={'naics_code': 'str'})
essential = essential[essential.essential]

In [14]:
patterns_joined = patterns_grouped.merge(essential, how='left', left_on='naics_4', right_on='naics_code')

In [ ]:
patterns_joined.to_csv("data/output/NAICS-visits-State.csv")

distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker


['/Users/raac/Development/covid/mobility-analysis/data/output/NAICS-visits-State.csv/0.part']

In [69]:
naics_essential = pd.read_csv("data/output/NAICS-visits-State.csv/0.part")
naics_essential = naics_essential.set_index('Unnamed: 0')
naics_essential = naics_essential.fillna(False)

,region,date,naics_4,raw_visit_counts,naics_code,essential
Unnamed: 0,,,,,,
0,AL,2019-12-31,4461,602451,4461,True
1,AL,2019-12-31,4523,1979675,4523,True
2,AL,2019-12-31,6113,501601,False,False
3,AL,2020-01-31,4461,540199,4461,True
4,AL,2020-01-31,4523,1890415,4523,True


In [70]:
naics_essential['essential'] = naics_essential['essential'].apply(lambda x: True if x == True else False, meta=('essential', 'bool'))

,Unnamed: 0,region,date,naics_4,raw_visit_counts,naics_code,essential
index,,,,,,,
0,0,AL,2019-12-31,4461.0,602451,4461.0,True
1,1,AL,2019-12-31,4523.0,1979675,4523.0,True
2,2,AL,2019-12-31,6113.0,501601,NaN,False
3,3,AL,2020-01-31,4461.0,540199,4461.0,True
4,4,AL,2020-01-31,4523.0,1890415,4523.0,True


In [80]:
output_grouped = naics_essential.groupby(['region', 'date', 'essential']).raw_visit_counts.sum().reset_index().compute()

In [81]:
output_total = naics_essential.groupby(['region', 'date']).raw_visit_counts.sum().reset_index().compute()
output_total.head()

,region,date,raw_visit_counts
0,AK,2019-12-31,1408351
1,AK,2020-01-31,1182579
2,AK,2020-02-29,885353
3,AL,2019-12-31,25137300
4,AL,2020-01-31,24040201


In [82]:
output_grouped = output_grouped.merge(output_total, how='left', on=['region', 'date']).rename(columns = {'raw_visit_counts_x': 'visits', 'raw_visit_counts_y': 'total_visits'})

In [84]:
output_grouped.to_csv("data/output/State-NAICS-Visits.csv")

tornado.application - ERROR - Uncaught exception GET /status (127.0.0.1)
HTTPServerRequest(protocol='http', host='localhost:8787', method='GET', uri='/status', version='HTTP/1.1', remote_ip='127.0.0.1')
Traceback (most recent call last):
  File "/Users/raac/Development/covid/mobility-analysis/venv/lib/python3.7/site-packages/tornado/web.py", line 1703, in _execute
  File "/Users/raac/Development/covid/mobility-analysis/venv/lib/python3.7/site-packages/bokeh/server/views/doc_handler.py", line 52, in get
  File "/Users/raac/Development/covid/mobility-analysis/venv/lib/python3.7/site-packages/bokeh/server/views/session_handler.py", line 112, in get_session
  File "/Users/raac/Development/covid/mobility-analysis/venv/lib/python3.7/site-packages/bokeh/util/token.py", line 112, in generate_jwt_token
  File "/Users/raac/Development/covid/mobility-analysis/venv/lib/python3.7/site-packages/bokeh/util/token.py", line 265, in _base64_encode
LookupError: unknown encoding: ascii
